In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import pydot
keras.utils.vis_utils.pydot = pydot
from keras import layers

In [2]:
device_name = tf.test.gpu_device_name()
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [11]:
# Read data and preprocess
df = pd.read_csv("data/rankings_100.csv")

df_labels = df.pop("score")
inputs = {}

for name, column in df.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = layers.StringLookup(vocabulary=np.unique(df[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

df_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

df_features_dict = {name: np.array(value) 
                         for name, value in df.items()}

features_dict = {name:values[:1] for name, values in df_features_dict.items()}
df_preprocessing(features_dict)


<tf.Tensor: shape=(1, 1820), dtype=float32, numpy=
array([[ 0.90489924, -0.08673687,  3.8444579 , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>

In [12]:
def mythic_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
  return model

mythic_model = mythic_model(df_preprocessing, inputs)


In [13]:
mythic_model.fit(x=df_features_dict, y=df_labels, epochs=10)

Epoch 1/10
63/63 [==============================] - 1s 2ms/step - loss: -309.3400
Epoch 2/10
63/63 [==============================] - 0s 2ms/step - loss: -1440.3727
Epoch 3/10
63/63 [==============================] - 0s 2ms/step - loss: -3547.0374
Epoch 4/10
63/63 [==============================] - 0s 2ms/step - loss: -6710.4453
Epoch 5/10
63/63 [==============================] - 0s 2ms/step - loss: -10941.8271
Epoch 6/10
63/63 [==============================] - 0s 2ms/step - loss: -16230.0439
Epoch 7/10
63/63 [==============================] - 0s 2ms/step - loss: -22567.2754
Epoch 8/10
63/63 [==============================] - 0s 2ms/step - loss: -29936.4883
Epoch 9/10
63/63 [==============================] - 0s 2ms/step - loss: -38306.2734
Epoch 10/10
63/63 [==============================] - 0s 2ms/step - loss: -47654.8633
